In [1]:
import os 
from dotenv import load_dotenv
load_dotenv

groq_api_key = os.getenv("GROQ_API_KEY")

In [59]:
from langchain_groq import ChatGroq
model = ChatGroq(model = "Gemma2-9b-It",groq_api_key = groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000015687984190>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000015687985060>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [60]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="hi, my name is rahul and I am a AI Engineer")])

AIMessage(content="Hello Rahul, it's nice to meet you! \n\nIt's great to know there are other AI Engineers out there. What kind of work do you do as an AI Engineer?  Are you working on any interesting projects right now? \n\nI'm always eager to learn more about what people are doing in the field of AI.\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 74, 'prompt_tokens': 22, 'total_tokens': 96, 'completion_time': 0.134545455, 'prompt_time': 0.000468388, 'queue_time': 0.100693766, 'total_time': 0.135013843}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-070af7be-a7a4-4081-b409-ae67e99df8ce-0', usage_metadata={'input_tokens': 22, 'output_tokens': 74, 'total_tokens': 96})

In [61]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="hi, my name is rahul and I am a AI Engineer"),
        AIMessage(content="Hello rahul!  it's nice to meet you!  \n\nIt's great to know there are other AI Engineers out there. What kind of AI work do you specialize in?  \n\nI'm always interested in learning more about how people are using AI to make a difference."),
        HumanMessage(content="Hey what's my name and what do I do"),
    ]
)

AIMessage(content="You said your name is Rahul, and you are an AI Engineer!  😊 \n\nIs there anything else you'd like to tell me about yourself or your work?  I'm happy to chat.  \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 47, 'prompt_tokens': 103, 'total_tokens': 150, 'completion_time': 0.085454545, 'prompt_time': 0.003391216, 'queue_time': 0.009324232000000002, 'total_time': 0.088845761}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-b8dd2a6c-252e-4316-a768-718b41d47690-0', usage_metadata={'input_tokens': 103, 'output_tokens': 47, 'total_tokens': 150})

In [62]:
###messsage history
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [63]:
config = {"configurable":{"session_id":"chat1"}}

In [64]:
response = with_message_history.invoke(
    [HumanMessage(content="hi, my name is rahul and I am a AI Engineer")],
    config=config,
)

response.content

"Hi Rahul, it's nice to meet you! \n\nIt's great to know you're an AI Engineer. What kind of work do you do in that field?  Are you working on any exciting projects right now?  \n\nI'm always interested in learning more about how people are using AI to make a difference.\n"

In [65]:
###changing the config----->session id 
config1 = {"configurable":{"session_id":"chat1"}}
response = with_message_history.invoke(
    [HumanMessage(content="what is my name?")],
    config=config1
)

response.content

'Your name is Rahul. 😊  \n\nI remember that from our first interaction.  \n\nIs there anything else I can help you with?\n'

In [66]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    ("system","you are helpful assistant. Answer all the questions to the best of your ability"),
    MessagesPlaceholder(variable_name="messages")
])

chain = prompt|model

In [67]:
chain.invoke({"messages":[HumanMessage(content = "hi! I'm rahul an AI Engineer ")]})

AIMessage(content="Hello Rahul! Nice to meet you.  \n\nI'm happy to answer your questions to the best of my ability. What can I do for you today? \n\nAs an AI assistant, I can help with things like:\n\n* **Generating creative content:**  Stories, poems, articles, dialogue, etc.\n* **Answering your questions:**  Based on the information I was trained on.\n* **Summarizing text:**  Give me a long piece of writing and I can provide a shorter version.\n* **Translating languages:**  I can translate between many different languages.\n* **And much more!**\n\nJust let me know what you need. 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 145, 'prompt_tokens': 35, 'total_tokens': 180, 'completion_time': 0.263636364, 'prompt_time': 0.000334299, 'queue_time': 0.013480681, 'total_time': 0.263970663}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-e8338bd1-1956-42f5-8fab-c1a0abb0e919-0',

In [77]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history)

In [96]:
config = {"configurable":{"session_id":"chat3"}}
response = with_message_history.invoke(
    [HumanMessage(content="hi, my name is rahul, imma a AI researcher")],
    config=config,
)

response.content

"Hi Rahul, it's great to meet you! \n\nBeing an AI researcher is fascinating work. What areas of AI research are you most interested in?  I'm always eager to learn more about the latest developments in the field.  😊 \n"

In [97]:
response = with_message_history.invoke(
    [HumanMessage(content="what is my name?")],
    config=config
)

response.content

'Your name is Rahul.  😊  \n\nIs there anything else I can help you with today, Rahul?'

In [98]:
response = with_message_history.invoke(
    [HumanMessage(content="what do i do")],
    config=config
)

response.content

"Since you're an AI researcher, Rahul, the possibilities are endless!  \n\nHere are some things you could do:\n\n* **Continue exploring your current research project:** Are you working on a specific problem or challenge in AI?  Dive deeper into it, analyze data, and experiment with new approaches.\n* **Learn about new AI techniques and algorithms:** The field is constantly evolving, so staying up-to-date is crucial. Read research papers, attend conferences, and explore online resources.\n* **Collaborate with other researchers:**  Connect with other AI enthusiasts, share your ideas, and work together on projects. \n* **Develop new AI applications:** Think about real-world problems that AI could solve and start building solutions.\n* **Teach and mentor others:** Share your knowledge and passion for AI with students or colleagues.\n\nWhat sparks your interest the most right now, Rahul?  I'm happy to brainstorm with you further! 😄 \n\n\n\n"

managing the conversation history

In [ ]:
from langchain_core.messages import SystemMessage,trim_messages
trimmed  = trim_messages(
    max_tokens = 70,
    strategy = "last",
    token_counter = model,
    include_system = True,
    allow_partial = False,
    start_on = "human"
)

messages = [
    SystemMessage(content="You are a helpful AI assistant."),
    HumanMessage(content="Hi, my name is Rahul."),
    AIMessage(content="Hello Rahul! How can I assist you today?"),
    HumanMessage(content="I'm working on a deep learning project involving RNNs and LSTMs. Can you help me?"),
    AIMessage(content="Of course! RNNs and LSTMs are powerful models for sequential data. What part of your project would you like help with?"),
    HumanMessage(content="I'm stuck on how to optimize my LSTM model. It's overfitting."),
    AIMessage(content="To combat overfitting in LSTMs, you can try techniques like adding dropout layers, reducing the complexity of the model, or using regularization. Would you like more details on any of these?")
]

trimmed.invoke(messages)

In [102]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

chain = (
    RunnablePassthrough.assign(messages = itemgetter("messages")|trimmed)|prompt|model
        )

chain.invoke(
    {
    "messages":messages+ [HumanMessage(content = "what project am i working on?")],
    "languages":"english"
    }
)

d:\Chatbot_using_LangChain\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
d:\Chatbot_using_LangChain\.venv\lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\rahul\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To 

AIMessage(content='As an AI, I have no memory of past conversations and no access to any information about you or your projects. To help me understand what you\'re working on, please tell me more about it! \n\nFor example, you could say:\n\n* "I\'m writing a short story about a robot who learns to feel emotions."\n* "I\'m building a website for my photography business."\n* "I\'m trying to learn how to code in Python."\n\n\nLet me know, and I\'ll do my best to assist you. 😊\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 118, 'prompt_tokens': 39, 'total_tokens': 157, 'completion_time': 0.214545455, 'prompt_time': 0.000372788, 'queue_time': 0.013585011999999999, 'total_time': 0.214918243}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-ba7d639b-b079-4450-bc33-e5c95c37e12e-0', usage_metadata={'input_tokens': 39, 'output_tokens': 118, 'total_tokens': 157})